In [67]:
from utils import *

"""
Calculate total and average fuel consumption for a given vessel and time range.
"""
def calculate_fuel_consumption(vessel_id, start_time, end_time):
    # Define the parameter ID for fuel consumption
    parameter_ids = ["enginemain_fuelcons_lph_1"]

    try:
        # Fetch data from PONTOS-HUB
        vessel_data = fetch_vessel_data(
            vessel_id = vessel_id,
            start_time = start_time,
            end_time = end_time,
            parameter_ids = parameter_ids,
            time_bucket = "1 minute"
        )

        # Convert fetched data into a Pandas DataFrame
        fuel_data_df = transform_vessel_data_to_dataframe(vessel_data)

        if fuel_data_df.empty:
            return {"message": "No data available for the specified parameters."}

        # Filter out non-zero fuel consumption
        non_zero_data = fuel_data_df[fuel_data_df["enginemain_fuelcons_lph_1"] > 0]

        if non_zero_data.empty:
            return {"message": "No non-zero data available for the specified parameters."}

        # Calculate the total and average fuel consumption
        total_fuel_consumption = non_zero_data["enginemain_fuelcons_lph_1"].sum().round(2)
        average_fuel_consumption = non_zero_data["enginemain_fuelcons_lph_1"].mean().round(2)

        return {
            "vessel_id": vessel_id,
            "time_frame": f"{start_time} to {end_time}",
            "total_fuel_consumption": total_fuel_consumption,
            "average_fuel_consumption": average_fuel_consumption,
        }
    except Exception as e:
        return {"message": f"An error occurred: {str(e)}"}

# Example usage
vessel_id = "mmsi_265558290"
start_time = "2024-11-01 00:00:00"
end_time = "2024-11-01 23:59:59"

result = calculate_fuel_consumption(vessel_id, start_time, end_time)
if "message" in result:
    print(result["message"])
else:
    print(f"Vessel ID: {result['vessel_id']}")
    print(f"Time Frame: {result['time_frame']}")
    print(f"Total Fuel Consumption: {result['total_fuel_consumption']} liters")
    print(f"Average Fuel Consumption per minute: {result['average_fuel_consumption']} liters")

Vessel ID: mmsi_265558290
Time Frame: 2024-11-01 00:00:00 to 2024-11-01 23:59:59
Total Fuel Consumption: 8256.23 liters
Average Fuel Consumption per minute: 10.36 liters


In [68]:
"""
Emission factors.
"""
EMISSION_FACTORS = {
    "diesel": 2.68,  # kg CO2 per liter
    "hvo": 1.89,     # kg CO2 per liter
    "lng": 2.75,     # kg CO2 per liter
    "electric": 0.0  # None
}

"""
Get the emission factor for a given fuel type.
"""
def get_emission_factor(fuel_type):
    return EMISSION_FACTORS.get(fuel_type.lower(), None)

In [69]:
def calculate_co2_emissions_dynamic(vessel_id, start_time, end_time, fuel_type):
    """
    Calculate total and average CO2 emissions dynamically for a vessel.
    """
    # Get the emission factor based on the fuel type
    emission_factor = get_emission_factor(fuel_type)

    if emission_factor is None:
        return {"message": f"Unknown fuel type: {fuel_type}. Please provide a valid fuel type."}

    # Fetch fuel consumption data using existing helper
    fuel_data = calculate_fuel_consumption(vessel_id, start_time, end_time)

    if "message" in fuel_data:
        return {"message": fuel_data["message"]}

    # Calculate CO2 emissions
    total_co2_emissions = (fuel_data["total_fuel_consumption"] * emission_factor).round(2)
    average_co2_emissions = (fuel_data["average_fuel_consumption"] * emission_factor).round(2)

    return {
        "vessel_id": vessel_id,
        "time_frame": fuel_data["time_frame"],
        "fuel_type": fuel_type,
        "emission_factor": emission_factor,
        "total_co2_emissions": total_co2_emissions,
        "average_co2_emissions_per_minute": average_co2_emissions
    }

# Example usage
vessel_id = "mmsi_265558290"
start_time = "2024-11-01 00:00:00"
end_time = "2024-11-01 23:59:59"
fuel_type = "diesel"

result = calculate_co2_emissions_dynamic(vessel_id, start_time, end_time, fuel_type)

if "message" in result:
    print(result["message"])
else:
    print(f"Vessel ID: {result['vessel_id']}")
    print(f"Time Frame: {result['time_frame']}")
    print(f"Fuel Type: {result['fuel_type']}")
    print(f"Emission Factor: {result['emission_factor']} kg CO2/L")
    print(f"Total CO2 Emissions: {result['total_co2_emissions']} kg")
    print(f"Average CO2 Emissions per minute: {result['average_co2_emissions_per_minute']} kg")

Vessel ID: mmsi_265558290
Time Frame: 2024-11-01 00:00:00 to 2024-11-01 23:59:59
Fuel Type: diesel
Emission Factor: 2.68 kg CO2/L
Total CO2 Emissions: 22126.7 kg
Average CO2 Emissions per minute: 27.76 kg
